In [2]:
from GroundProjection import *
from IntersectArea import *
from ColumnProjection import *
# from shapely.geometry import Polygon

from 求太阳各角度 import *
from 阳光向量 import *
from 求镜面顶点 import *
from 求镜面法向量 import *

import numpy as np


In [3]:
print(GroundProjection(np.array([1,2,3]),np.array([-1,-3,-4])))

[1.75 0.   6.  ]


In [13]:
vector = np.array([200,0,76])
points = np.array([[-197.4,3,2.5],[-197.4,-3,2.5],[-202.60,-3,5.5],[-202.60,3,5.5]])
print(ColumnProjection(vector,points)[:,[0,2]])

a
[[   0.       3.      77.512]
 [-197.4     -3.       2.5  ]
 [-202.6     -3.       5.5  ]
 [-202.6      3.       5.5  ]]
[[   0.       3.      77.512]
 [   0.      -3.      77.512]
 [-202.6     -3.       5.5  ]
 [-202.6      3.       5.5  ]]
[[ 0.00000000e+00  3.00000000e+00  7.75120000e+01]
 [ 0.00000000e+00 -3.00000000e+00  7.75120000e+01]
 [-2.84217094e-14 -3.00000000e+00  8.24880000e+01]
 [-2.02600000e+02  3.00000000e+00  5.50000000e+00]]
[[ 0.00000000e+00  3.00000000e+00  7.75120000e+01]
 [ 0.00000000e+00 -3.00000000e+00  7.75120000e+01]
 [-2.84217094e-14 -3.00000000e+00  8.24880000e+01]
 [-2.84217094e-14  3.00000000e+00  8.24880000e+01]]
[[ 3.    77.512]
 [-3.    77.512]
 [-3.    82.488]
 [ 3.    82.488]]


In [ ]:
def e_mirror_cut_rate(phi,day,hour,mirror_center_point):
    '''
        单面镜子 集热塔截断面积占比  截断效率
        
        维度phi 时间day hour  -- 太阳照射角度
        镜子中心坐标，  -- 镜子法向量&四角坐标
    '''

    # 计算镜子投影位置并旋转
    sun_vector = get_sun_vector(get_alpha_s(phi,day,hour),get_gamma_s(phi,day,hour))
    mirror_column = get_mirror_out_vector(mirror_center_point)
    mirror_points = get_mirror_point(mirror_center_point,get_mirror_normal_vector(mirror_center_point,sun_vector,mirror_column))
    x_points = ColumnProjection(mirror_column,mirror_points)
    
    # 坐标二维化
    points = x_points[:,[0,2]]

    # 集热塔面积
    tower = np.array([3.5,76],[3.5,84],[-3.5,84],[-3.5,76])
    
    # 计算截断面积
    cut_area = IntersectArea(tower,points)

    # 计算投射总面积
    
